In [0]:
%sql
create database if not exists deltadb;
use deltadb

In [0]:
schema="order_id long, order_date string, order_customer_id long, order_status string"
orders=spark.read\
    .format("csv")\
    .schema(schema)\
    .option('header','true')\
    .option("path","/FileStore/tables/orders/orders.csv")\
    .load()

In [0]:
orders1=orders.limit(10)
orders1.write\
    .format("delta")\
    .mode("overwrite")\
    .option("path","/FileStore/tables/delta/orders")\
    .save()

In [0]:
orders.write\
    .format("delta")\
    .mode("overwrite")\
    .option("path","/FileStore/tables/delta/orders_full")\
    .save()

In [0]:
%sql
create table if not exists deltadb.orders using delta location "/FileStore/tables/delta/orders";
create table if not exists deltadb.orders_full using delta location "/FileStore/tables/delta/orders_full";

In [0]:
%sql
insert into deltadb.orders select * from deltadb.orders_full order by rand() limit 100;

num_affected_rows,num_inserted_rows
100,100


In [0]:
spark.sql("select * from deltadb.orders").show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|   39619|2014-03-26 00:00:...|             5424|     PROCESSING|
|   31379|2014-02-04 00:00:...|             5368|       COMPLETE|
|   42466|2014-04-12 00:00:...|              893|        PENDING|
|   24746|2013-12-25 00:00:...|            11628|PENDING_PAYMENT|
|   26561|2014-01-05 00:00:...|            10969|        ON_HOLD|
|   16441|2013-11-05 00:00:...|            11143|        ON_HOLD|
|   39640|2014-03-27 00:00:...|             1212|       COMPLETE|
|   34264|2014-02-20 00:00:...|            10480|PENDING_PAYMENT|
|   40392|2014-03-31 00:00:...|             6284|     PROCESSING|
|   12043|2013-10-06 00:00:...|             1762|       COMPLETE|
|   16698|2013-11-06 00:00:...|             4528|       COMPLETE|
|   22782|2013-12-11 00:00:...|             6193|       COMPLETE|
|   44971|

In [0]:
%sql
update deltadb.orders
set order_status='COMPLETE'
where order_id= 39619

num_affected_rows
1
